In [8]:
## 신경망 정의하기

import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 입력 이미지 채널 1개, 출력 채널 6개, 5x5의 정사각 컨볼루션 행렬
        # 컨볼루션 커널 정의
        self.conv1 = nn.Conv2d(1, 6, 5)     # 입력 채널: 1, 출력 채널: 6, 커널(합성곱 필터) 크기: 5x5
        self.conv2 = nn.Conv2d(6, 16, 5)    # 입력 채널: 6, 출력 채널: 16, 커널 크기: 5x5
        # 완전 연결 레이어 (Affine 연산). 아핀(affine) 연산: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)   # 5*5은 이미지 차원에 해당. 입력 크기: 16x5x5, 출력 크기: 120
        self.fc2 = nn.Linear(120, 84)           # 입력 크기: 120, 출력 크기: 84
        self.fc3 = nn.Linear(84, 10)            # 입력 크기: 84, 출력 크기: 10

    def forward(self, input):
        # 합성곱(Convolution) 레이어 c1: 입력 이미지 채널 1, 출력 채널 6,
        # 5x5 정사각 합성곱, 활성 함수로 RELU 사용 및 (N, 6, 28, 28)의 크기를 갖는 Tensor를 출력 (N은 배치 크기)
        c1 = F.relu(self.conv1(input))
        # 서브샘플링(Subsampling) 레이어 s2: 2x2 격자, 순전히 기능적인 레이어로,
        # 이 레이어는 어떠한 매개변수도 가지지 않고 (N, 6, 14, 14) Tensor를 출력
        s2 = F.max_pool2d(c1, (2, 2))
        # 합성곱(Convolution) 레이어 c3: 입력 채널 6, 출력 채널 16,
        # 5x5 정사각 합성곱, 활성 함수로 RELU 사용 및 (N, 16, 10, 10)의 크기를
        # 갖는 Tensor를 출력
        c3 = F.relu(self.conv2(s2))
        # 서브샘플링(Subsampling) 레이어 s4: 2x2 격자, 순전히 기능적인 레이어로,
        # 이 레이어는 어떠한 매개변수도 가지지 않고 (N, 16, 5, 5) Tensor를 출력
        s4 = F.max_pool2d(c3, 2)
        # 평탄화(flatten) 연산: 순전히 기능적으로 동작하며, (N, 400) Tensor를 출력
        s4 = torch.flatten(s4, 1)
        # 완전히 연결된 레이어 f5: (N, 400) Tensor를 입력으로 받아서
        # (N, 120) Tensor를 출력하며, 활성 함수로 RELU 사용
        f5 = F.relu(self.fc1(s4))
        # 완전히 연결된 레이어 f6: (N, 120) Tensor를 입력으로 받아서
        # (N, 84) Tensor를 출력하며, 활성 함수로 RELU 사용
        f6 = F.relu(self.fc2(f5))
        # 가우시안 레이어 출력: (N, 84) Tensor를 입력으로 받아서
        # (N, 10) Tensor를 출력
        output = self.fc3(f6)
        return output

    def forward(self, x):
        # (2, 2) 크기 윈도우에 대해 맥스 풀링(max pooling)
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # 크기가 제곱수라면, 하나의 숫자만을 특정(specify)
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = torch.flatten(x, 1) # 배치 차원을 제외한 모든 차원을 하나로 평탄화(flatten)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [9]:
# forward 함수만 정의하고 나면, (변화도를 계산하는) backward 함수는 autograd 를 사용하여 자동으로 정의됩니다.
# forward 함수에서는 어떠한 Tensor 연산을 사용해도 됩니다.

# 모델의 학습 가능한 매개변수들은 net.parameters() 에 의해 반환됩니다.

params = list(net.parameters())
print(len(params))
print(params[0].size())  # conv1의 .weight

10
torch.Size([6, 1, 5, 5])


In [10]:
# 임의의 32x32 입력값을 넣어보겠습니다.

# 이 신경망(LeNet)의 예상되는 입력 크기는 32x32입니다. 이 신경망에 MNIST 데이터셋을 사용하기 위해서는, 데이터셋의 이미지 크기를 32x32로 변경해야 합니다.

input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[-0.1379, -0.1185,  0.0286, -0.0722, -0.0086, -0.0256,  0.0075, -0.0367,
          0.0952,  0.0860]], grad_fn=<AddmmBackward0>)


In [11]:
# 모든 매개변수의 변화도 버퍼(gradient buffer)를 0으로 설정하고, 무작위 값으로 역전파를 합니다:

net.zero_grad()
out.backward(torch.randn(1, 10))

In [12]:
## 손실 함수

# 손실 함수는 (output, target)을 한 쌍(pair)의 입력으로 받아, 출력(output)이 정답(target)으로부터 얼마나 멀리 떨어져 있는지 추정하는 값을 계산합니다.
# nn 패키지에는 여러가지의 손실 함수들 이 존재합니다. 간단한 손실 함수로는 출력과 대상간의 평균제곱오차(mean-squared error)를 계산하는 nn.MSEloss 가 있습니다.

output = net(input)
target = torch.randn(10)  # 예시를 위한 임의의 정답
target = target.view(1, -1)  # 출력과 같은 shape로 만듦
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(0.9836, grad_fn=<MseLossBackward0>)


In [13]:
# 이제 .grad_fn 속성을 사용하여 loss 를 역방향에서 따라가다 보면, 이러한 모습의 연산 그래프를 볼 수 있습니다:

# input -> conv2d -> relu -> maxpool2d -> conv2d -> relu -> maxpool2d
#       -> flatten -> linear -> relu -> linear -> relu -> linear
#       -> MSELoss
#       -> loss

# 따라서 loss.backward() 를 실행할 때, 전체 그래프는 신경망의 매개변수에 대해 미분되며,
# 그래프 내의 requires_grad=True 인 모든 Tensor는 변화도가 누적된 .grad Tensor를 갖게 됩니다.

# 설명을 위해, 역전파의 몇 단계를 따라가 보겠습니다

print(loss.grad_fn)  # MSELoss
print(loss.grad_fn.next_functions[0][0])  # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLU

In [14]:
## 역전파(Backprop)

# 오차(error)를 역전파하기 위해서는 loss.backward() 만 해주면 됩니다.
# 기존에 계산된 변화도의 값을 누적 시키고 싶지 않다면 기존에 계산된 변화도를 0으로 만드는 작업이 필요합니다.

# 이제 loss.backward() 를 호출하여 역전파 전과 후에 conv1의 bias 변수의 변화도를 살펴보겠습니다.

net.zero_grad()     # 모든 매개변수의 변화도 버퍼를 0으로 만듦

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
None
conv1.bias.grad after backward
tensor([-0.0035,  0.0061, -0.0163,  0.0210,  0.0323,  0.0021])


In [16]:
## 가중치 갱신

# 실제로 많이 사용되는 가장 단순한 갱신 규칙은 확률적 경사하강법(SGD; Stochastic Gradient Descent)입니다

# 새로운 가중치 = 가중치 - 학습률 * 변화도
# weight = weight - learning_rate * gradient

learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)



In [17]:
# 신경망을 구성할 때 SGD, Nesterov-SGD, Adam, RMSProp 등과 같은 다양한 갱신 규칙을 사용하고 싶을 수 있습니다.
# 이를 위해서 torch.optim 라는 작은 패키지에 이러한 방법들을 모두 구현해두었습니다. 사용법은 매우 간단합니다

import torch.optim as optim

# Optimizer를 생성합니다.
optimizer = optim.SGD(net.parameters(), lr=0.01)

# 학습 과정(training loop)은 다음과 같습니다:
optimizer.zero_grad()   # 변화도 버퍼를 0으로
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()    # 업데이트 진행